<a href="https://colab.research.google.com/github/Tirta2241720045/PCVK_Ganjil_2024/blob/main/Week12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nama: Tirta Nurrochman Bintang Prawira
# NIM: 2241720045
# Kelas: TI-3A
# No Absen: 27

In [1]:
pip install tensorflow

## Langkah 1 - Import Library

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import numpy as np
import os
from tensorflow.keras import layers, models

## Langkah 2 - Pra Pengolahan Data

### Langkah 2.1. Pra Pengolahan Data Training

In [10]:
# Direktori untuk data pelatihan
train_dir = '/content/drive/MyDrive/dataset/Minggu12/Dataset_Daging_Hewan_Ternak/Training'

# Augmentasi gambar untuk data pelatihan
train_datagen = ImageDataGenerator(
    rescale=1./255,          # Normalisasi gambar
    rotation_range=20,       # Rotasi acak
    width_shift_range=0.2,   # Pergeseran lebar acak
    height_shift_range=0.2,  # Pergeseran tinggi acak
    shear_range=0.2,         # Pemotongan acak
    zoom_range=0.2,          # Zoom acak
    horizontal_flip=True,    # Pembalikan horizontal acak
    fill_mode='nearest'      # Isi ruang kosong
)

# Membaca dan mempersiapkan data pelatihan dengan ukuran gambar yang konsisten
train_generator = train_datagen.flow_from_directory(
    train_dir,               # Direktori data pelatihan
    target_size=(224, 224),  # Ukuran gambar yang digunakan, pastikan sama dengan input model
    batch_size=32,
    class_mode='categorical' # Karena ini adalah klasifikasi multikelas
)

Found 240 images belonging to 3 classes.


### Langkah 2.2. Pra Pengolahan Data Testing

In [11]:
test_dir = '/content/drive/MyDrive/dataset/Minggu12/Dataset_Daging_Hewan_Ternak/Testing'
test_datagen = ImageDataGenerator(rescale=1./255)

# Membaca dan mempersiapkan data pengujian dengan ukuran gambar yang konsisten
test_generator = test_datagen.flow_from_directory(
    test_dir,               # Direktori data pengujian
    target_size=(224, 224), # Ukuran gambar yang digunakan
    batch_size=32,
    class_mode='categorical' # Karena ini adalah klasifikasi multikelas
)

Found 60 images belonging to 3 classes.


## Langkah 3 - Pembuatan Model CNN

In [12]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),  # Sesuaikan ukuran input
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),  # Dropout untuk mengurangi overfitting
    layers.Dense(train_generator.num_classes, activation='softmax')  # Output layer
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy', # Menggunakan categorical crossentropy untuk klasifikasi multikelas
              metrics=['accuracy'])

## Langkah 4 - Fit CNN

In [14]:
# Melatih model dengan data pelatihan dan validasi
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size
)

KeyboardInterrupt: 

## Langkah 5 - Prediksi dengan 1 Citra

In [ ]:
# Path gambar yang akan diprediksi
img_path = '/content/drive/MyDrive/dataset/Minggu12/Dataset Daging Hewan Ternak/single_prediction/your_image.jpg'

# Memuat gambar dan melakukan pra-pemrosesan
img = load_img(img_path, target_size=(224, 224))  # Mengubah ukuran gambar
img_array = img_to_array(img) / 255.0  # Normalisasi gambar
img_array = np.expand_dims(img_array, axis=0)  # Menambah dimensi untuk batch

# Melakukan prediksi
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions, axis=1)  # Mengambil kelas dengan probabilitas tertinggi

# Menampilkan hasil prediksi dan persentase
class_labels = list(train_generator.class_indices.keys())  # Daftar label kelas (Ayam, Kambing, Sapi)
predicted_label = class_labels[predicted_class[0]]

# Menampilkan persentase untuk setiap kelas
for i, class_name in enumerate(class_labels):
    print(f"{class_name}: {predictions[0][i] * 100:.2f}%")

print(f"\nPredicted class: {predicted_label}")

## Langkah 6 - Penyimpanan Model


In [ ]:
# Menyimpan model
model.save('/content/drive/MyDrive/dataset/Minggu12/model_daging.h5')